In [24]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from scipy import optimize
from scipy import integrate
from scipy.signal import find_peaks
from scipy.integrate import simpson
from scipy.integrate import quad
from pathlib import Path
from os import listdir, chdir
from os.path import isfile, join
import regex as re
from lmfit import Model
from lmfit.models import LinearModel, GaussianModel, ExponentialModel, ConstantModel, PowerLawModel, PolynomialModel, LorentzianModel, VoigtModel
import math
import time
import itertools as it
%matplotlib inline

In [2]:
def make_dataframe(sample_name, data_path):
    
    #INPUT - Set the path to the output files from the I vs q script - Use absolute path
    #data_path = r'C:\Users\Elizabeth\APS Spring 2022\Data Processing\Test_data\Sample6b_charge_dbl\Ivsq_text'
    os.chdir(data_path)
    
    # Importing integrated XRD pattern from APS synchrotron expierment in Fall 2021 at beamline 5-BM-C
    file = open(sample_name)
    data = pd.read_csv(file, skiprows = 13, header = None, delim_whitespace=True)
    df = pd.DataFrame(data)
    df.columns = ['q','I']
    #display(df)
        
    return df

In [3]:
def get_xy_motor(sample_name, data_path):

    try:
        # Find the x_motor position in the file title using Regex
        start_x = re.search('_x_', sample_name).end()
        end_x = re.search('mm_ss_stg2_y_', sample_name).start()
        x_motor = sample_name[start_x:end_x].replace(',', '.')
        x_motor = float(x_motor)

        # Find the y_motor position in the file title using Regex
        start_y = re.search('_y_', sample_name).end()
        end_y = re.search('mm_primary', sample_name).start()
        y_motor = sample_name[start_y:end_y].replace(',', '.')
        y_motor = float(y_motor)
    
    except AttributeError:
        print('oh shit bra, the name changed! (function could not find x and y position in file name)')
        x_motor = input('Whats the x value?')
        x_motor = float(x_motor)
        
        y_motor = input('Whats the y value?')
        y_motor = float(y_motor)
        print("Groovie.")
    
    return x_motor, y_motor

#x_motor, y_motor = get_xy_motor(sample_name, data_path)

In [4]:
def get_points(df,q_min,q_max):
    
    ''' This function creates a condensed dataframe that isolates the deired peak
    Inputs: data set in data frame (df), lower q bound for peak(q_min), upper q bound for peak(q_max)
    Outputs: shortened dataframe (df_cut)'''
    
    return df[(df['q'] >= q_min) & (df['q'] <= q_max)]

#df_cut = get_points(df,q_min,q_max)

In [5]:
def lmfit_model(df_cut, q1_guess, q2_guess, data_path, x_motor, y_motor, peak_name, plot, sample_name):
    x = df_cut['q']
    y = df_cut['I']
    
    b = 'background'
    p1 = 'peak1'
    p2 = 'peak2'
    
    #initial guesses     
    slope1 = 0 
    int1 = 12
    
    #initial guess conditions for peak 1 
    q1 = q1_guess
    q1min = q1 - 0.01
    q1max = q1 + 0.02 
    sig = 0.01
    amp = 1
    
    #initial guess conditions for peak 2 
    q2 = q2_guess
    q2min = q2 - 0.01
    q2max = q2 + 0.02 

    background = LinearModel(prefix=(b + '_'))  
    pars = background.make_params()
    
    # For linear background
    pars = background.make_params()
    pars[b + '_slope'].set(slope1)
    pars[b + '_intercept'].set(int1)
    
    
    
    # Gaussian peak 1
    peak1 = GaussianModel(prefix= (p1 + '_'))
    pars.update(peak1.make_params())
    pars[p1+'_center'].set(q1, max = q1max, min = q1min)
    pars[p1+'_sigma'].set(sig, max = 0.05)
    pars[p1+'_amplitude'].set(amp, min = 0)
    
    # Gaussian peak 2
    peak2 = GaussianModel(prefix= (p2 + '_'))
    pars.update(peak2.make_params())
    pars[p2+'_center'].set(q2, max = q2max, min = q2min) 
    pars[p2+'_sigma'].set(sig, max = 0.05)
    pars[p2+'_amplitude'].set(amp, min = 0)
    
    model = peak1 + peak2 + background
    
    result = model.fit(df_cut['I'], pars, x = df_cut['q'], nan_policy = 'omit')
    
    comps = result.eval_components(x=x) 

    #plot 1
    if plot == True: 
        fig, ax = plt.subplots(1,1, figsize=(7,7))
        ax.plot(x,y, label='Data')
        ax.plot(x,result.best_fit, label='Model')
        ax.plot(x, comps[b+'_'], '--', label='Linear')
        ax.plot(x, comps[p1+'_'], '--', label='Gaussian1')
        ax.plot(x, comps[p2+'_'], '--', label='Gaussian2')

        ax.set_xlim(q_min, q_max)
        ax.set_title('Graphite, LixC6: ('+ str(x_motor) + ',' + str(y_motor) + ')\n' + sample_name) 
        ax.set_xlabel('q [1/A]')
        ax.set_ylabel('I [au.]')
        ax.legend()
    
    model = result.best_fit
    
    # Peak1 fit
    Gaussian1 = comps[p1+'_']
    fwhm1 = result.params[p1+'_fwhm'].value
    center1 = result.params[p1+'_center'].value
    
    #Peak2 fit
    Gaussian2 = comps[p2+'_']
    fwhm2 = result.params[p2+'_fwhm'].value
    center2 = result.params[p2+'_center'].value
    
    return Gaussian1, fwhm1, center1, Gaussian2, fwhm2, center2

#result = lmfit_model(df_cut, q_min, q_max)
#print(result)6y6yy^


In [6]:
def make_model(q_max, q_min, model_centers, sig, amp):
    background = LinearModel(prefix=('b' + '_'))  
    pars = background.make_params()
    
    model = background
    
    # initial guesses     
    slope1 = 0 
    int1 = 12
    
    # For linear background
    pars = background.make_params()
    pars['b' + '_slope'].set(slope1)
    pars['b' + '_intercept'].set(int1)
    
    for peak, center in enumerate(model_centers):
        # create prefex for each peak
        pref = 'g'+str(peak)+'_'
        peak = GaussianModel(prefix=pref)
        # set the parimiters for each peak
        pars.update(peak.make_params())
        pars[pref+'center'].set(value=center, min=q_min, max=q_max)
        pars[pref+'sigma'].set(value=sig, max = 0.05)
        pars[pref+'amplitude'].set(amp, min = 0)
        
        model = model + peak

    return (model, pars)
        

In [7]:
def get_model_list(df_cut, q_max, q_min, num_of_centers, num_peaks, sig, amp):
    # set some inital parimiters
    
    # generate a list of centers to try
    increment = (q_max - q_min) / num_of_centers
    n = 0
    center_list = []
    
    while n <= num_of_centers:
        center_list.append(n*increment + q_min)
        n += 1
    q_range = q_max - q_min
    center_list[0] = center_list[0] + .1 * q_range
    # -1 refers to the last element in the list
    center_list[-1] = center_list[-1] - .1 * q_range
    
    # creat unique combination of peak positions returns a list of tuples. 
    # Tuples are samp length of num_peaks
    center_list = list(it.combinations(center_list, num_peaks))
    
    # make a list of models for each center
    model_list = []
    for center in center_list:
        model_list.append(make_model(q_max, q_min, center, sig, amp))
    
    return(model_list)  
        
    # result = model.fit(df_cut['I'], pars, x = df_cut['q'], nan_policy = 'omit')
    # mod_results = 
    
    

In [8]:
def run_model(df_cut, model, pars):
    model_result = model.fit(df_cut['I'], pars, x = df_cut['q'], nan_policy = 'omit')
    return(model_result)

In [9]:
def user_model(best_model, df_cut, sig, amp):
    good = 'no'
    print("\n\n fit not found")
    best_model.plot()
    plt.pause(1)
    while good != 'yes':  
        try:
            centers =  input('Enter peak centers separated by space \n')
            centers = centers.split(',')
            for i in range(len(centers)):
                # convert each item to int type
                centers[i] = float(centers[i])
            print(centers)
            # make_model(q_max, q_min, model_centers, sig, amp):
            model = make_model(q_max, q_min, centers, sig, amp)
            best_model = run_model(df_cut, model[0], model[1])
            print("chisqr is ", best_model.chisqr)
            best_model.plot()
            plt.pause(1)
            good = input('enter yes to continue. To try again enter no\n')
        except:
            print('operation filed with the following messege')
    return best_model

In [10]:
def fit_data(df_cut, q_max, q_min, num_of_centers, sig, amp):
    chisqr = 10000
    num_peaks = 1
    while chisqr >= 3:
        
        if num_peaks >= 4:
            print('turn user_model on')
            #best_model = user_model(best_model, df_cut, sig, amp)
            return best_model
            
        # returns a list of tuples. first value is the model second value is the pars. 
        # looks like this ((model, pars), (model, pars), ........)
        model_list = get_model_list(df_cut, q_max, q_min, num_of_centers, num_peaks, sig, amp)
        
        model_result_list = []

        for i in range(len(model_list)):
            model = model_list[i][0]
            pars = model_list[i][1]
            model_result_list.append(run_model(df_cut, model, pars))
            
        results_sorted = sorted(model_result_list, key=lambda model: model.chisqr)
        best_model = results_sorted[0]
        chisqr = best_model.chisqr
        num_peaks += 1
        
    #best_model.plot()
    plt.pause(1)
    return best_model

In [11]:
def get_values(best_model, df_cut):
         
    # a list of tuples with 4 values. the peak data, fwhm, and center.
    # Looks like ((peak_data, fwhm, center, guess), (peak_data, fwhm, center, guess), ........)
    comps_list = []
    comps = best_model.eval_components(x=df_cut['q'])
    for prefex in comps.keys():
        if prefex != 'b_':
            comps_list.append(((comps[str(prefex)]), best_model.params[str(prefex)+'fwhm'].value, best_model.params[str(prefex)+'center'].value, 1.75))
    
    
    integral_list = []
    fwhm_peak_center_list = []
    
    for vals in comps_list:
        integral_val = integrate_model(df_cut, vals[0], vals[2], vals[3])
        integral_list.append(integral_val)
        fwhm_peak_center_list = get_fwhm_center(integral_val, vals[1], vals[2], vals[3])
        
    return integral_list, fwhm_peak_center_list
    

In [12]:
def master_function(read_sample_file, num_of_centers,  data_path, q_min, q_max, peak_name, sample_name, sig, amp):
    
    # Make a dataframe of the entire XRD pattern
    df = make_dataframe(read_sample_file, data_path)
    # Get xy_motor positions
    x_motor, y_motor = get_xy_motor(read_sample_file, data_path)
    
    # Slice the dataframe to desired q range
    df_cut = get_points(df, q_min, q_max)

    # get the best fit for the data
    best_model = fit_data(df_cut, q_max, q_min, num_of_centers, sig, amp)

    if best_model is not None:
        integral_list, fwhm_peak_center_list = get_values(best_model, df_cut)
    else:
        return

    
    # Use lmfit to generate peak model
    #(Gaussian1, fwhm1_raw, center1_raw, Gaussian2, fwhm2_raw, center2_raw) = lmfit_model(df_cut, 
    #    q1_guess, q2_guess, data_path, x_motor, y_motor, peak_name, plot, read_sample_file)
    
    
    
#     #Integrate the model numerically 
#     integral1 = integrate_model(df_cut, Gaussian1, center1_raw, q1_guess)
#     integral2 = integrate_model(df_cut, Gaussian2, center2_raw, q2_guess)
    
#     #Get FWHM and Peak Center
#     fwhm1, center1 = get_fwhm_center(integral1, fwhm1_raw, center1_raw, q1_guess)
#     fwhm2, center2 = get_fwhm_center(integral2, fwhm2_raw, center2_raw, q2_guess)
    
    return [sample_name, x_motor, y_motor, integral_list, fwhm_peak_center_list, best_model]

In [13]:
def get_fwhm_center(integral, fwhm_raw, center_raw, q_guess): 
        
    if integral > 0.05 and center_raw < q_guess + 0.05 and center_raw > q_guess - 0.05:
        fwhm = fwhm_raw
        center = center_raw
    else:
        fwhm = float("nan") 
        center = float("nan") 
    return fwhm, center

In [14]:
def integrate_model(df_cut, Gaussian, center_raw, q_guess):
    
    # Define model
    Model = Gaussian
    
    # Select the data to integrate over
    q_range = df_cut['q'].tolist()
    

    # Caclulate the integral based on the direct data using Simpson's rule
    integral = integrate.simpson(Model, q_range, even='avg')
    
    if integral > 0.05 and center_raw < q_guess + 0.05 and center_raw > q_guess - 0.05:
        return integral
    else: 
        integral = float("nan")
        return integral

#integral = integrate_model3(df_cut, result)
#print(integral)

In [26]:

startTime = time.time()

# Sample info
sample_name = 'Sample9'
peak_name = 'graphite_LixC6'

#Set isolated peak q range
q_min = 1.75
q_max = 1.9

# #Peak loccation guesses
# q1_guess = 1.69
# q2_guess = 1.87

# numper of centers to try
num_of_centers = 5

#inital guesses for sig and amplatude
sig = 0.05 
amp = 5

#Setup dataframe 
df_integrals = pd.DataFrame(columns=['Sample','x motor', 'y motor','Gaussian1', 'FWHM1', 'Center1',
                                     'Gaussian2', 'FWHM2', 'Center2'])

# Set path to correct (where the I vs q data is)
# data_path_gen = r'D:\xpdUser_Allan-Cole_309400_d56e2a5d_2022-07-04-1443\tiff_base'
# data_path = data_path_gen + '\\' + sample_name + '\\integration'
# os.chdir(data_path)

data_path = r'C:\Users\Elizabeth Allan-Cole\Desktop\XRD Data Processing\NSLS-II Winter 2023\Planning\Ben - test cases\Sample9_map_charge'


# Make a list of all files names in folder
list_of_files = [files for files in listdir(data_path) if isfile(join(data_path, files))]

n = 0
# loop through the list of files and append df_integrals --> Troubleshoot the peak fitting, getting weird numbers! 
for i in range(10,200,2): 
    if 'mean_q' in list_of_files[i]:
        #print("\nnew file start\n")
        #Call the master function to get the integral values for the specified peak
        # returns [sample_name, x_motor, y_motor, integral_list, fwhm_peak_center_list, best_model]
        get_integrals = master_function(list_of_files[i], num_of_centers, data_path, q_min, q_max, peak_name, sample_name, sig, amp)
        print(n)
        n += 1

        #Save plot images
        savePath_gen = r"C:\Users\Elizabeth Allan-Cole\Desktop\XRD Data Processing\NSLS-II Winter 2023\Planning\test pics"
        #os.path.join(path, "User/Desktop", "file.txt")
        savePath = os.path.join(savePath_gen, sample_name, 'Model_plots', str(get_integrals[1]), str(get_integrals[2]), str(list_of_files[i]), ".pdf")
        #savePath = savePath_gen + r'\' + sample_name + 'Model_plots' + str(get_integrals[1]) + '_' + str(get_integrals[2])
        #+ str(list_of_files[i]) + ".pdf"
        print(savePath)
        #get_integrals[5].plot().savefig(savePath)
        plt.savefig(savePath + '/' + (list_of_files[i] + '_' + peak_name +".pdf"))
        break
        #Fill in dataframe columns    
       # df_integrals.loc[i,] = get_integrals
        
# print(n)    
# #Set path to save integral file - Use absolute path - (Where intergral folder will be stored)
# integral_folder_path_gen = r'C:\Users\Elizabeth Allan-Cole\Desktop\XRD Data Processing\NSLS-II Summer 2022\Data Processing\Integral Output'
# integral_folder_path = integral_folder_path_gen + '\\' + sample_name
# os.chdir(integral_folder_path)

# # Turn dataframe into csv file
# #integral_file = df_integrals.to_csv(integral_folder_path + '\\' + sample_name + '_' + peak_name + '.csv' , index=False)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))# Sample info

0
C:\Users\Elizabeth Allan-Cole\Desktop\XRD Data Processing\NSLS-II Winter 2023\Planning\test pics\Sample9\Model_plots\30.0\66.0\Sample9_20220701-141343_907cae_sample_x_30,00mm_ss_stg2_y_66,00mm_primary-51_mean_q.chi\.pdf


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Elizabeth Allan-Cole\\Desktop\\XRD Data Processing\\NSLS-II Winter 2023\\Planning\\test pics\\Sample9\\Model_plots\\30.0\\66.0\\Sample9_20220701-141343_907cae_sample_x_30,00mm_ss_stg2_y_66,00mm_primary-51_mean_q.chi\\.pdf/Sample9_20220701-141343_907cae_sample_x_30,00mm_ss_stg2_y_66,00mm_primary-51_mean_q.chi_graphite_LixC6.pdf'

<Figure size 432x288 with 0 Axes>

In [ ]:
    
    # get number of peaks
    # num_peaks = find_peaks(list(df_cut['I']), threshold =.05, prominence = .5)
    avg_height = sum(df_cut['I'])/len(df_cut)
    num_peaks = find_peaks(list(df_cut['I']), prominence = .3, width = .01, height = avg_height)
    print('peaks return value')
    print(num_peaks[0])
    for x in num_peaks[0]:
        print(x, list(df_cut['I'])[x], list(df_cut['q'])[x], avg_height)

In [ ]:
def master_function(read_sample_file, num_of_centers,  data_path, q_min, q_max, peak_name, plot, sample_name):
    
    # Make a dataframe of the entire XRD pattern
    df = make_dataframe(read_sample_file, data_path)
    # Get xy_motor positions
    x_motor, y_motor = get_xy_motor(read_sample_file, data_path)
    
    # Slice the dataframe to desired q range
    df_cut = get_points(df, q_min, q_max)


    # TODO loop this till fit is good
    chisqr = 10000
    num_peaks = 1
    while chisqr >= 2:
        
        if num_peaks >= 4:
            print("\n\n fit not found \n\n")
            best_model.plot()
            break
            
        # returns a list of tuples. first value is the model second value is the pars. 
        # looks like this ((model, pars), (model, pars), ........)
        model_list = get_model_list(df_cut, q_max, q_min, num_of_centers, num_peaks, sig = 0.05, amp = 5)
        
        model_result_list = []

        for i in range(len(model_list)):
            model = model_list[i][0]
            pars = model_list[i][1]
            model_result_list.append(run_model(df_cut, model, pars))
            
        results_sorted = sorted(model_result_list, key=lambda model: model.chisqr)
        best_model = results_sorted[0]
        chisqr = best_model.chisqr
        num_peaks += 1
        
    best_model.plot()
    
        
        
    #TODO add filter to se if fit is good here 
    # a list of tuples with 4 values. the peak data, fwhm, and center.
    # Looks like ((peak_data, fwhm, center, guess), (peak_data, fwhm, center, guess), ........)
    comps_list = []
    comps = best_model.eval_components(x=df_cut['q'])
    for prefex in comps.keys():
        if prefex != 'b_':
            comps_list.append(((comps[str(prefex)]), best_model.params[str(prefex)+'fwhm'].value, best_model.params[str(prefex)+'center'].value, 1.75))
    
    
    integral_list = []
    fwhm_peak_center_list = []
    
    for vals in comps_list:
        integral_val = integrate_model(df_cut, vals[0], vals[2], vals[3])
        integral_list.append(integral_val)
        fwhm_peak_center_list = get_fwhm_center(integral_val, vals[1], vals[2], vals[3])
    
    
    # Use lmfit to generate peak model
    #(Gaussian1, fwhm1_raw, center1_raw, Gaussian2, fwhm2_raw, center2_raw) = lmfit_model(df_cut, 
    #    q1_guess, q2_guess, data_path, x_motor, y_motor, peak_name, plot, read_sample_file)
    
    
    
#     #Integrate the model numerically 
#     integral1 = integrate_model(df_cut, Gaussian1, center1_raw, q1_guess)
#     integral2 = integrate_model(df_cut, Gaussian2, center2_raw, q2_guess)
    
#     #Get FWHM and Peak Center
#     fwhm1, center1 = get_fwhm_center(integral1, fwhm1_raw, center1_raw, q1_guess)
#     fwhm2, center2 = get_fwhm_center(integral2, fwhm2_raw, center2_raw, q2_guess)
    
    return sample_name, x_motor, y_motor, integral_list, fwhm_peak_center_list